<a href="https://colab.research.google.com/github/OdysseusPolymetis/colabs_for_nlp/blob/main/2_analyse_textuelle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ANALYSE TEXTUELLE : tokénisation, lemmatisation, syntaxe et entités nommées**

In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=50a07897175c7dba649df42a6dbd259a14c29f8f4dcc2e94369b4e8e60bd59e7
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


##Qu'est-ce que c'est que l'analyse textuelle (automatique), rapidement
En fait ça peut toucher beaucoup de domaines. Ici, on va en voir plusieurs, entre autres la **tokenisation**, la **lemmatisation**, le **postagging**. J'appellerai ça les étapes de "pre-processing", de travail préliminaire.
<br>En effet, on a très souvent besoin, pour faire des choses plus poussées, de ces étapes pour éviter de créer des biais dans les analyses qui vont suivre.
<br>Voici quelques exemples de biais possibles, si l'on ne passe pas par certaines de ces étapes de pré-traitement.

Prenons l'exemple des nuages de mots. C'est globalement facile à faire, et ça permet souvent (de manière parfois un peu artificielle) d'illustrer son propos.
Ici voyons ce qu'on obtient si on fait un nuage de mots sur les _Trois Mousquetaires_.

![](https://drive.google.com/uc?export=view&id=1g8RLOXodDDGBgfR6tJ-jvVG1JHTdz65f)

Ce nuage est fait avec des données brutes. C'est-à-dire qu'on a juste injecté les mots du texte, tels quels. Globalement, on ne peut pas en faire grand chose et ce n'est pas représentatif. Pourquoi ? Déjà parce que chaque mot est encore fléchi, et donc pour chaque "fut", "est", "sera", chaque mot est compté comme un mot à part, et non comme une forme de "être".
<br>Une première solution est donc de **lemmatiser**.

![](https://drive.google.com/uc?export=view&id=10vv3sdsqLMBMXYW5-YAKp9ztgYq_sJDs)

C'est déjà mieux, mais ça n'est pas fou non plus. Il y a encore de nombreux mots dont on se fiche complètement. Exemple, "faire", "plus", etc.
<br>On a donc encore une étape, celle d'enlever les **mots-outils**, ou "stopwords", et la ponctuation. Les mots-outils sont les mots qui n'ont qu'un faible poids sémantique (qui peuvent avoir leur importance pour une étude dédiée, certes), mais qui font du bruit lors d'une analyse statistique de base.
<br>Enlevons-les.

![](https://drive.google.com/uc?export=view&id=1c9PxvrkXDTNZTC7QOGNCi2qOQ5_fI9th)

##Comment on pré-traite ?
Il y a plusieurs écoles, et donc aussi plusieurs modules pour le faire.
<br>D'expérience personnelle, il y a les modules rapides et qui donnent des résultats plus mitigés, et des modules plus longs qui sont généralement meilleurs (mais pas forcément).
<br>Ici, je vais vous montrer plusieurs outils avec des qualités et des défauts, pour le français (mais ils ont aussi des modèles en d'autres langues).

##**Quelques outils en ligne utiles**

###**UDPipe**
<br>Vous le trouverez [ici](https://lindat.mff.cuni.cz/services/udpipe/).
<br>C'est un bon outil pour un texte court. Dès que vous allez faire un texte intégral ou un peu long, ça rame très vite.
<br>Mais vous avez des fonctionnalités sympathiques, type les arbres syntaxiques en images vectorielles.**texte en gras**

###**Deucalion**
<br>Vous le trouverez [ici](https://dh.chartes.psl.eu/deucalion/api/fr/).
<br>Bien plus gérable pour les textes longs. Une sortie pas sexy, mais ce n'est pas ce qu'on lui demande. Il est surtout très efficace pour des langues plus rares, type l'ancien français ou le grec et le latin.

###**VoyantTools**
<br>Vous le trouverez [ici](https://voyant-tools.org/).
<br>C'est un outil de visualisation. C'est pratique et un peu shiny, mais on peut faire des choses très développées dessus.

#**LE TAL : TOKENISATION, LEMMATISATION, POSTAGGING**

Nous allons tester un outil principal, qui permet de faire ces trois actions, **`stanza`**. Il en existe beaucoup d'autres (notamment spacy et pie-extended), mais stanza est le plus performant en termes de rapport qualité/temps.

In [ ]:
preface="Tant qu’il existera, par le fait des lois et des mœurs, une damnation sociale créant artificiellement, en pleine civilisation, des enfers, et compliquant d’une fatalité humaine la destinée qui est divine ; tant que les trois problèmes du siècle, la dégradation de l’homme par le prolétariat, la déchéance de la femme par la faim, l’atrophie de l’enfant par la nuit, ne seront pas résolus ; tant que, dans de certaines régions, l’asphyxie sociale sera possible ; en d’autres termes, et à un point de vue plus étendu encore, tant qu’il y aura sur la terre ignorance et misère, des livres de la nature de celui-ci pourront ne pas être inutiles."

Essayons maintenant sur un fichier plus long. Ici je télécharge le même fichier des _Misérables_.

In [ ]:
!gdown --id 1GEgd5cQoJkTm5PRWfixxOKHe3uOlxFqo

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1GEgd5cQoJkTm5PRWfixxOKHe3uOlxFqo
To: /content/miserables.txt
100% 3.17M/3.17M [00:00<00:00, 213MB/s]


In [ ]:
filepath_of_text = "/content/miserables.txt"

In [ ]:
full_text = open(filepath_of_text, encoding="utf-8").read()

##**stanza (précédemment Stanford CoreNLP)**

Personnellement, c'est l'analyseur que j'utilise quand je n'ai pas besoin de faire de représentations graphiques de mes résultats. Il est rapide et efficace.

Je mentionne `stanza` en particulier pour trois raisons :
<br>- d'abord parce qu'il dispose d'un très grand nombre de modèles de langue, et pas forcément des langues très répandues,
<br>- ensuite parce qu'il est très rapide, et niveau performance tout à fait satisfaisant pour les gros corpus,
<br>- enfin parce que je le trouve facile à manipuler et à implémenter.

Mais il faut garder en tête qu'il en existe bien d'autres qui fonctionnent vraiment très bien, avec un nombre de modèles qui se multiplie. Je pense au tagger de BERT,  ou de `flair` entre autres. Mais ça nécessite d'être un peu plus aguerri.

Là encore, il existe plusieurs modèles rien que pour le français (je vous mets ici la [liste des modèles français](https://universaldependencies.org/fr/index.html), et ici la [liste des modèles](https://stanfordnlp.github.io/stanza/available_models.html) dans d'autres langues), mais le modèle par défaut peut être appelé avec `fr`.
<br>C'est un modèle sur du français moderne, donc ne fonctionne pas avec l'ancien français etc. (mais un modèle existe aussi pour l'ancien français, appelé par défaut avec `fro`).

In [ ]:
import stanza
stanza.download('fr')

INFO:stanza:Downloading default packages for language: fr (French) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


On commence par lui spécifier une Pipeline, c'est-à-dire qu'on lui signifie quels processeurs il va devoir mobiliser pour les opérations suivantes et en quelle langue. L'avantage de cette opération est qu'on ne mobilise pas l'artillerie lourde quand on veut faire des opérations simples).

In [ ]:
nlp_stanza = stanza.Pipeline(lang='fr', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


Maintenant, nous pouvons lancer le TAL sur l'ensemble du texte (il est beaucoup plus rapide que `spacy` à cet égard).

In [ ]:
miserables_stanza=nlp_stanza(full_text)

Déjà, premier constat, l'étiqueteur est plus efficace en terme de temps que `spacy` (il peut analyser l'intégralité des _Misérables_ dans un temps tout à fait correct (moins de deux minutes). Mais on va le sauvegarder quand même.

Voyons maintenant ses résultats.

In [ ]:
for sent in miserables_stanza.sentences[50:60]:
  for token in sent.words:
    print(token.text + ' - ' + token.lemma + ' - ' + token.pos)

et - et - CCONJ
, - , - PUNCT
en - en - ADP
vieillissant - vieillir - VERB
, - , - PUNCT
elle - il - PRON
avait - avoir - AUX
gagné - gagner - VERB
ce - ce - PRON
qu' - que - PRON
on - on - PRON
pourrait - pouvoir - VERB
appeler - appeler - VERB
la - le - DET
beauté - beauté - NOUN
de - de - ADP
la - le - DET
bonté - bonté - NOUN
. - . - PUNCT
Ce - ce - PRON
qui - qui - PRON
avait - avoir - AUX
été - être - AUX
de - de - ADP
la - le - DET
maigreur - maigreur - NOUN
dans - dans - ADP
sa - son - DET
jeunesse - jeunesse - NOUN
était - être - AUX
devenu - devenir - VERB
, - , - PUNCT
dans - dans - ADP
sa - son - DET
maturité - maturité - NOUN
, - , - PUNCT
de - de - ADP
la - le - DET
transparence - transparence - NOUN
; - ; - PUNCT
et - et - CCONJ
cette - ce - DET
diaphanéité - diaphanéité - NOUN
laissait - laisser - VERB
voir - voir - VERB
l' - le - DET
ange - ange - NOUN
. - . - PUNCT
C' - ce - PRON
était - être - AUX
une - un - DET
âme - âme - NOUN
plus - plus - ADV
encore - encore - AD

Globalement ça marche d'enfer. Et c'est de très loin le plus rapide (et le plus récent).

# **LES ENTITES NOMMEES**

Les **entités nommées** sont des éléments textuels connus et catégorisables. Par exemple, un NER (Named Entity Recognition) qui fonctionnerait bien comprendrait "la ville aux sept collines" comme un nom de lieu, et comme Rome. 

##**stanza**

**`stanza`** aussi peut permettre de façon beaucoup plus rapide d'obtenir les NER. Ne serait-ce que parce que, par défaut, il demande de définir une Pipeline (un processus qui permet de ne pas faire toutes les opérations en même temps).

In [ ]:
stanza_ner = stanza.Pipeline(lang='fr', processors='tokenize,ner')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| ner       | wikiner |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


Je reprends le même passage que celui donné à `spacy` précédemment.

In [ ]:
doc = stanza_ner("qui avaient couru sur son compte au moment où il était arrivé dans le diocèse. Vrai ou faux, ce qu'on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu'ils font. M. Myriel était fils d'un conseiller au parlement d'Aix ; noblesse de robe.")

In [ ]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

entity: Myriel	type: PER
entity: Aix	type: LOC


Les résultats sont meilleurs pour ce passage, semble-t-il.
<br>Si on fait la recherche sur l'ensemble du roman, voilà ce que stanza trouve (attention, l'exécution est un peu longue, vous avez le `pickle` deux cellules plus bas).

In [ ]:
ents_stanza = stanza_ner(full_text)

In [ ]:
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in ents_stanza.ents], sep='\n')

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
entity: Liberté_	type: LOC
entity: faubourg Saint-Antoine	type: LOC
entity: faubourg Saint-Antoine	type: LOC
entity: Le coup qu'elle frappe	type: MISC
entity: Cette	type: LOC
entity: faubourg Saint-Antoine	type: LOC
entity: Paris	type: LOC
entity: Antoine_	type: PER
entity: faubourg Antoine	type: LOC
entity: Mont Aventin	type: LOC
entity: Ennius	type: PER
entity: faubourg Saint-Antoine	type: LOC
entity: Cette	type: LOC
entity: Saint-Antoine	type: PER
entity: Paris	type: LOC
entity: Moyen-Âge	type: LOC
entity: Aucune	type: LOC
entity: Dieu	type: PER
entity: Chapitre VI	type: MISC
entity: Enjolras	type: PER
entity: Enjolras	type: PER
entity: café Musain	type: LOC
entity: Enjolras	type: PER
entity: Si	type: MISC
entity: Ceux	type: LOC
entity: Cette	type: LOC
entity: Courfeyrac	type: LOC
entity: Feuilly	type: LOC
entity: la Glacière	type: LOC
entity: Combeferre	type: PER
entity: Picpus	type: LOC
entity: Bahorel	t